# Section 2.2 Data Structure Activities

In [43]:
import pymc3 as pm
import arviz as az
import pandas as pd
np.random.seed(0)

## Reproducing the Planet Experiment
More good news! Your astronomical discovery, from Section 1.2, has been published, but now people want to you to share your data and results. They also are asking for help getting seeing portions of your analysis runs to inspect in greater detail.

### Exercise 1 
Your favorite PPL is PyMC3, but it turns out your peer reviewer likes Stan. In an alternate universe your favorite PPL is stan, but now your peer reviewer is a PyMC3 gal. Here we introduce the *Law of researcher PPL choice*  
  
$$P(\text{Your friends uses another PPL} | \text{You choice of PPL}) = 1$$


**How can we use ArviZ, Xarray, and NetCDF to share results in a common way?**  
Note: We encourage you to use whatver PPL you prefer . These docs may come in helpful  
https://arviz-devs.github.io/arviz/api.html#data

#### Step 1: Define your model and generate results

In [5]:
observations = [0, 0, 1, 0, 1]
water_observations = sum(observations)
total_observations = len(observations)

In [6]:
with pm.Model() as planetmodel:
    p_water = pm.Uniform("p", 0 ,1)
    w = pm.Binomial("w", p=p_water, n=total_observations, observed=water_observations)
    trace = pm.sample(5000, chains=2)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [p]
Sampling 2 chains: 100%|██████████| 11000/11000 [00:05<00:00, 1843.73draws/s]


#### Step 2: Convert model results from PPL to Az.InferenceData

In [7]:
water_data = az.from_pymc3(trace=trace)

/Users/canyon/.miniconda3/envs/bayestutorial/lib/python3.6/site-packages/arviz/data/io_pymc3.py:56: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  chain_likelihoods.append(np.stack(log_like))


#### Step 3: Inspect InferenceData to see what groups exist

In [8]:
water_data

Inference data with groups:
	> posterior
	> sample_stats
	> observed_data

#### Step 4: Inspect Posterior group to verify variables count, chain count, and draw count

In [9]:
water_data.posterior

<xarray.Dataset>
Dimensions:  (chain: 2, draw: 5000)
Coordinates:
  * chain    (chain) int64 0 1
  * draw     (draw) int64 0 1 2 3 4 5 6 7 ... 4993 4994 4995 4996 4997 4998 4999
Data variables:
    p        (chain, draw) float64 0.2129 0.1732 0.1503 ... 0.1973 0.3375 0.3728
Attributes:
    created_at:                 2019-04-27T16:23:35.654539
    inference_library:          pymc3
    inference_library_version:  3.6

#### Step 3: Save your model to disk

In [10]:
water_data.to_netcdf("WaterResults.nc")

'WaterResults.nc'

### Exercise 2
You've been asked to peer review a study on radon levels in Minnesota basements (Find Reference). Your colleague provides you a NetCDF file of her analysis and results, located at in the data repository of this notebook. 

You've been asked to do a couple things.

#### Load the NetCDF file into python memory

In [11]:
radon_data = az.from_netcdf("../../data/radon.nc")

#### List all the groups
See what analysis your colleague has already run by checking the groups present in the InferenceData object

In [12]:
radon_data

Inference data with groups:
	> posterior
	> sample_stats
	> posterior_predictive
	> prior
	> observed_data

#### Count the number of counties
How many counties were included in the observed_data?
Hint: xarray has a `.to_dataframe()` method

In [16]:
radon_data.observed_data

<xarray.Dataset>
Dimensions:          (observed_county: 919)
Coordinates:
  * observed_county  (observed_county) object 'AITKIN' ... 'YELLOW MEDICINE'
Data variables:
    y_like           (observed_county) float64 ...
Attributes:
    created_at:                 2018-10-05T15:29:14.882229
    inference_library:          pymc3
    inference_library_version:  3.5

#### How many variables are in Bayesian model?
Inspect the posterior xarray dataset and get a list of all variables in the model.

In [49]:
radon_data.posterior

<xarray.Dataset>
Dimensions:          (chain: 4, county: 85, draw: 500, gamma_dim_0: 3, observed_county: 919)
Coordinates:
  * chain            (chain) int64 0 1 2 3
  * draw             (draw) int64 0 1 2 3 4 5 6 ... 493 494 495 496 497 498 499
  * gamma_dim_0      (gamma_dim_0) int64 0 1 2
  * county           (county) object 'AITKIN' 'ANOKA' ... 'YELLOW MEDICINE'
  * observed_county  (observed_county) object 'AITKIN' ... 'YELLOW MEDICINE'
Data variables:
    gamma            (chain, draw, gamma_dim_0) float64 ...
    eps_a            (chain, draw, county) float64 ...
    b                (chain, draw) float64 ...
    sigma_a          (chain, draw) float64 ...
    mu_a             (chain, draw, observed_county) float64 ...
    a                (chain, draw, observed_county) float64 ...
    sigma_y          (chain, draw) float64 ...
Attributes:
    created_at:                 2018-10-05T15:29:14.514378
    inference_library:          pymc3
    inference_library_version:  3.5

#### Select first 10 values of chain 2 for sigma_y in the posterior
Using the `.sel` method get the first ten values 

In [52]:
radon_data.posterior.sel(chain=[2], draw=slice(0,10))["sigma_y"]

<xarray.DataArray 'sigma_y' (chain: 1, draw: 11)>
array([[0.748223, 0.748223, 0.738897, 0.742252, 0.749223, 0.743694, 0.760952,
        0.760952, 0.760952, 0.760952, 0.711894]])
Coordinates:
  * chain    (chain) int64 2
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 9 10